In [14]:
!pip install geopy
!pip install folium
!pip install bs4
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import folium
from folium import plugins

import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

from sklearn.cluster import KMeans 

print("libraries imported")

libraries imported


## Part 1 - Webscraping, Data Wrangling, and Cleaning


In [15]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969' 

data = requests.get(url).text 

In [16]:
w_soup = BeautifulSoup(data, 'html5lib')

In [17]:
# Scrape the table in the Wikipedia page for the PostalCode, Borough, and Neighborhood

# Setup lists for postal_code, borough, and neighborhood
post_list = []
boro_list = []
neighb_list = []

tab_bodies = w_soup.find_all('tbody')

tab_data = tab_bodies[0]

for i, row in enumerate(tab_data.find_all('tr')):
    if i == 0:
        headers = row.find_all('th')
        postal_header = headers[0].getText().strip()
        borough_header = headers[1].getText().strip()
        neighborhood_header = headers[2].getText().strip()
    else:
        cols = row.find_all('td')
        postal_code = cols[0].getText()
        borough = cols[1].getText()
        neighborhood = cols[2].getText()
        
        post_list.append(postal_code.strip())
        boro_list.append(borough.strip())
        neighb_list.append(neighborhood.strip())

In [18]:
# Create pandas dataframe containing postal_code, borough, neighborhood information

toronto_data = {postal_header : post_list,
               borough_header : boro_list,
               neighborhood_header : neighb_list}

tor_df = pd.DataFrame(toronto_data)
tor_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [19]:
# Replace 'Not assigned' to NaN
tor_df['Borough'].replace('Not assigned', np.nan, inplace = True)
tor_df.shape

(180, 3)

In [20]:
# Drop cells with NaN

tor_df.dropna(subset = ['Borough'], axis = 0, inplace = True)

In [21]:
# Sort the dataframe by postal code 
tor_df.sort_values(by=['Postal Code'], inplace = True)
tor_df.reset_index(drop = True, inplace=True)
tor_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
tor_df.shape

(103, 3)